In [1]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"
os.environ["HUMANLAYER_API_KEY"]="hl-a-4fnbHdLc0QNpRTEbPZ6Mu4Gjw2D2uFILDckbyNI-HDQ"
os.environ["QWEN_API_KEY"] = "sk-4c461e0b5a3f4de1a2ad3ce436f2b0cc"

In [2]:
os.environ.get("QWEN_API_KEY")

'sk-4c461e0b5a3f4de1a2ad3ce436f2b0cc'

In [3]:
from humanlayer.core.approval import HumanLayer

In [4]:
hl = HumanLayer(verbose=True)

/root/miniconda3/lib/python3.11/site-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


In [5]:
# add can be called without approval
def add(x: int, y: int) -> int:
    """Add two numbers together."""
    return x + y


# but multiply must be approved by a human
@hl.require_approval()
def multiply(x: int, y: int) -> int:
    """multiply two numbers"""
    return x * y

In [6]:
from camel.toolkits import FunctionTool
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

In [7]:
model = ModelFactory.create(
    model_platform=ModelPlatformType.QWEN,
    model_type=ModelType.QWEN_QWQ_32B,
)

tools = [FunctionTool(add), FunctionTool(multiply)]

agent_with_tools = ChatAgent(
    model = model,
    tools=tools
)

# Interact with the agent
response = agent_with_tools.step("multiply 2 and 5, then add 32 to the result")
print("\n\n----------Result----------\n\n")
print(response.msgs[0].content)

/root/work/camel/camel/toolkits/function_tool.py:461: UserWarning: Parameter description is missing for 
                            {'type': 'integer'}. This may affect the quality of tool 
                            calling.
  warnings.warn(f"""Parameter description is missing for


HumanLayer: waiting for approval for multiply via humanlayer cloud
HumanLayer: human approved multiply


----------Result----------


The multiplication of 2 and 5 is 10. Now, I need to add 32 to this result. So, 10 plus 32 equals 42.


Human-in-loop interaction

In [8]:
from camel.toolkits import HumanToolkit
human_toolkit = HumanToolkit()

In [9]:
model = ModelFactory.create(
    model_platform=ModelPlatformType.QWEN,
    model_type=ModelType.QWEN_MAX,
)
agent = ChatAgent(
    system_message="You are a helpful assistant.",
    model=model,
    tools=[*human_toolkit.get_tools()],
)

response = agent.step(
    "Test me on the capital of some country, and comment on my answer."
)

print(response.msgs[0].content)

Question: What is the capital of France?
That's correct! The capital of France is indeed Paris. Good job! 🎉
